In [3]:
ATTRIBUTES = {"race": 1,
              "gender": 2,
              "age": 3,
              "skintone": 4,
              "emotion": 5,
              "masked": 6}
attribute_name = "race"
attrs = ATTRIBUTES[attribute_name]
print(attrs)

1
